In [ ]:
# label_count as scores for Varierr dataset

from pathlib import Path
import json

input_path = Path("/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr.json")
output_file = Path("/Users/phoebeeeee/ongoing/Beyond-noise/dataset/chaosnli_label_count.json")

id_to_negative_label_count = {}

label_to_short = {
    "contradiction": "c",
    "entailment": "e",
    "neutral": "n"
}

with input_path.open("r", encoding="utf-8") as f:
    for line in f:
        instance = json.loads(line)
        instance_id = instance["id"]
        chaos_counts = instance.get("chaosnli_labels", {})
        round1_counts = instance.get("label_count_round_1", {})

        for full_label, count in round1_counts.items():
            if count is None:
                continue  
            short_label = label_to_short[full_label]
            chaos_count = chaos_counts.get(short_label, 0)  

            key = f"{instance_id}-{short_label}"
            id_to_negative_label_count[key] = float(f"{-0.01 * chaos_count:.2f}")

with output_file.open("w", encoding="utf-8") as f:
    json.dump(id_to_negative_label_count, f, indent=1)

print(f"✅ Clean ChaosNLI label count scores saved to {output_file}")


✅ Clean ChaosNLI label count scores saved to /Users/phoebeeeee/ongoing/Beyond-noise/dataset/chaosnli_label_count.json


{"id": "61429c", "context": "In this enclosed but airy building, you'll find ladies with large machetes expertly chopping off hunks of kingfish, tuna, or shark for eager buyers.", "statement": "You'll find small lepers chopping of chunks of tuna, its the only place they can work.", "entailment": [], "neutral": [{"annotator": 0, "id": "1265-neutral-1", "judgments": [{"annotator": 0, "makes_sense": true}, {"annotator": 1, "makes_sense": true}, {"annotator": 2, "makes_sense": true}, {"annotator": 3, "makes_sense": true}], "label_correction": false, "reason": "The context doesn't mention whether the ladies are small lepers and whether its the only place they can work.", "self_corrected": false}, {"annotator": 1, "id": "1265-neutral-2", "judgments": [{"annotator": 0, "makes_sense": true}, {"annotator": 1, "makes_sense": true}, {"annotator": 2, "makes_sense": true}, {"annotator": 3, "makes_sense": true}], "label_correction": false, "reason": "The context does not say anything about lepers or where they could work.", "self_corrected": false}, {"annotator": 2, "id": "1265-neutral-3", "judgments": [{"annotator": 0, "makes_sense": true}, {"annotator": 1, "makes_sense": true}, {"annotator": 2, "makes_sense": true}, {"annotator": 3, "makes_sense": true}], "label_correction": false, "reason": "\"Small lepers\" don't have to be \"ladies\"; we don't know whether \"small lepers\" can find other jobs.", "self_corrected": false}, {"annotator": 3, "id": "1265-neutral-4", "judgments": [{"annotator": 0, "makes_sense": true}, {"annotator": 1, "makes_sense": true}, {"annotator": 2, "makes_sense": true}, {"annotator": 3, "makes_sense": true}], "label_correction": false, "reason": "Lepers and the only place to work at are not mentioned", "self_corrected": false}], "contradiction": [], "idk": [], "label_count_round_1": {"contradiction": null, "entailment": null, "neutral": 4.0}, "label_count_round_2": {"contradiction": null, "entailment": null, "neutral": 4.0}, "label_set_round_1": ["neutral"], "label_set_round_2": ["neutral"], "error_labels": [], "has_ambiguity": false, "chaosnli_labels": {"n": 41, "c": 57, "e": 2}}


In [65]:
from pathlib import Path
import json

varierr_path = Path("/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr.json")  
llm_input_path = Path("/Users/phoebeeeee/ongoing/Beyond-noise/2_llm_validation/341/llama_no_label-guided/llama_elaborate/with_deduplication/with_validation_count.json")
output_file = Path("/Users/phoebeeeee/ongoing/Beyond-noise/2_llm_validation/341/llama_no_label-guided/llama_elaborate/with_deduplication/llama3.2_label_count.json")

label_to_short = {
    "contradiction": "c",
    "entailment": "e",
    "neutral": "n"
}

id_to_llm_counts = {}
with llm_input_path.open("r", encoding="utf-8") as f:
    for line in f:
        instance = json.loads(line)
        instance_id = instance["id"]
        counts = instance.get("label_count_round_1", {})
        id_to_llm_counts[instance_id] = counts

id_to_negative_label_count = {}
with varierr_path.open("r", encoding="utf-8") as f:
    for line in f:
        instance = json.loads(line)
        instance_id = instance["id"]
        label_counts = instance.get("label_count_round_1", {})

        for full_label, count in label_counts.items():
            if count is None:
                continue 

            short_label = label_to_short[full_label]
            llm_counts = id_to_llm_counts.get(instance_id, {})
            llm_label_count = llm_counts.get(short_label, 0)

            key = f"{instance_id}-{short_label}"
            id_to_negative_label_count[key] = float(f"{-1 * llm_label_count:.2f}")

output_file.parent.mkdir(parents=True, exist_ok=True)
with output_file.open("w", encoding="utf-8") as f:
    json.dump(id_to_negative_label_count, f, indent=1)

print(f"Saved to {output_file}")


Saved to /Users/phoebeeeee/ongoing/Beyond-noise/2_llm_validation/341/llama_no_label-guided/llama_elaborate/with_deduplication/llama3.2_label_count.json


In [66]:
# eval main

import json
import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict
from sklearn.metrics import average_precision_score, precision_score, recall_score

VARIERR_PATH = Path("/Users/phoebeeeee/ongoing/Beyond-noise/dataset/varierr.json")
SCORE_PATH = Path("/Users/phoebeeeee/ongoing/Beyond-noise/2_llm_validation/500/human_exp+human_dist+LLM_validate/avg_llama3.1_validate_varierr_scores.json")

def recall_at_k(y_true, y_score, k):
    y_true = np.array(y_true)
    y_score = np.array(y_score)
    idx = np.argsort(y_score)[::-1][:k]
    y_pred = np.zeros_like(y_true)
    y_pred[idx] = 1
    return recall_score(y_true, y_pred)

def precision_at_k(y_true, y_score, k):
    y_true = np.array(y_true)
    y_score = np.array(y_score)
    idx = np.argsort(y_score)[::-1][:k]
    return precision_score(y_true[idx], np.ones(k))

def compute_metrics(id_to_score, id_to_gt):
    ids = [i for i in id_to_gt if i in id_to_score]
    y_true = np.array([id_to_gt[i] for i in ids])
    y_score = np.array([id_to_score[i] for i in ids])

    return {
        "ap": average_precision_score(y_true, y_score),
        "p@50": precision_at_k(y_true, y_score, 50),
        "p@100": precision_at_k(y_true, y_score, 100),
        "r@100": recall_at_k(y_true, y_score, 100)
    }

def prettify_results(df: pd.DataFrame) -> str:
    df *= 100
    header = "Method\tAP\tP@50\tP@100\tR@100"
    row = df.round(1).iloc[0]
    values = "\t".join([f"${v:.1f}$" for v in row])
    return f"{header}\n{df.index[0]}\t{values}"

def get_ground_truth(instances, strategy="label_correction", rerank_single_label=False):
    id_to_gt = {}
    for ex in instances:
        id_ = ex["id"]
        round1 = ex.get("label_count_round_1", {})
        errors = ex.get("error_labels", [])
        label_set_round_2 = ex.get("label_set_round_2", [])

        for label, count in round1.items():
            if count is None or (rerank_single_label and count > 1):
                continue
            label_id = f"{id_}-{label[0]}"
            if strategy == "label_correction":
                id_to_gt[label_id] = label in errors
            elif strategy == "ambiguous":
                id_to_gt[label_id] = (len(label_set_round_2) > 1) and (label in label_set_round_2)
            elif strategy == "label_correction_or_ambiguous":
                label_correction = label in errors
                ambiguous = (len(label_set_round_2) > 1) and (label in label_set_round_2)
                id_to_gt[label_id] = label_correction or ambiguous
    return id_to_gt

def build_score_table(varierr_path, score_path, strategy="label_correction", rerank_single_label=False):
    with open(varierr_path) as f:
        data = [json.loads(line) for line in f]

    with open(score_path) as f:
        id_to_score = json.load(f)

    id_to_gt = get_ground_truth(data, strategy=strategy, rerank_single_label=rerank_single_label)
    id_to_score = {k: v for k, v in id_to_score.items() if k in id_to_gt}

    metrics = compute_metrics(id_to_score, id_to_gt)

    df = pd.DataFrame([metrics], index=[score_path.stem])
    return prettify_results(df)

print(build_score_table(
    varierr_path=VARIERR_PATH,
    score_path=SCORE_PATH,
    strategy="label_correction"
))


Method	AP	P@50	P@100	R@100
avg_llama3.1_validate_varierr_scores	$24.2$	$28.0$	$19.0$	$14.7$


In [53]:
id_to_score

{'23751e-e': 0.9,
 '23751e-c': 0.8,
 '61429c-n': 0.2875,
 '54811c-n': 0.775,
 '54811c-c': 0.7,
 '12601n-n': 0.7625,
 '38477c-c': 0.7999999999999999,
 '1735n-n': 0.8,
 '1735n-c': 0.85,
 '7449e-e': 0.2,
 '7449e-n': 0.6,
 '24385e-n': 0.75,
 '49807n-e': 0.8500000000000001,
 '49807n-n': 0.7333333333333334,
 '138448c-n': 0.8,
 '138448c-c': 0.8500000000000001,
 '48454c-n': 0.675,
 '48454c-c': 0.8,
 '73260n-e': 0.9,
 '73260n-n': 0.8000000000000002,
 '73260n-c': 0.8,
 '76219n-e': 0.9,
 '76219n-n': 0.4,
 '10229n-n': 0.7125,
 '99791n-e': 0.95,
 '99791n-n': 0.5333333333333333,
 '13964e-e': 0.875,
 '13964e-n': 0.7,
 '66185c-e': 0.925,
 '66185c-n': 0.5,
 '66185c-c': 0.9,
 '65066n-n': 0.6625,
 '76020e-e': 0.9,
 '76020e-c': 0.8333333333333334,
 '83248c-n': 0.8,
 '83248c-c': 0.8666666666666667,
 '79141n-e': 0.8833333333333333,
 '79141n-n': 0.825,
 '97926n-e': 0.8,
 '97926n-n': 0.75,
 '45957e-n': 0.7,
 '45957e-c': 0.5333333333333333,
 '24126n-e': 0.9,
 '24126n-n': 0.5,
 '135251n-e': 0.9,
 '135251n-n': 0

In [42]:
# check top 30 items
with open(SCORE_PATH) as f:
    id_to_score = json.load(f)

with open(VARIERR_PATH) as f:
    data = [json.loads(line) for line in f]

id_to_gt = get_ground_truth(data, strategy="label_correction")

id_to_score = {k: v for k, v in id_to_score.items() if k in id_to_gt}

scored = [(k, -v, id_to_gt[k]) for k, v in id_to_score.items()]

scored_sorted = sorted(scored, key=lambda x: x[1], reverse=True)

print("\nTop 30 sorted items (after -score):")
print(f"{'id_label':<20} {'-score':<10} {'GT(error)':<10}")
for k, neg_score, gt in scored_sorted[:30]:
    print(f"{k:<20} {neg_score:<10.3f} {gt}")



Top 30 sorted items (after -score):
id_label             -score     GT(error) 
19578n-c             -0.000     False
83657c-c             -0.000     False
49462c-c             -0.000     True
52542n-c             -0.000     True
47408n-n             -0.000     True
131235c-c            -0.000     False
34776c-c             -0.000     True
135247c-c            -0.000     False
89995c-e             -0.000     True
87332c-e             -0.000     True
72870c-e             -0.000     True
127073c-c            -0.000     True
95883e-c             -0.000     False
26495e-c             -0.000     False
53619c-e             -0.000     True
82528c-e             -0.000     True
112402c-c            -0.000     False
54383c-c             -0.000     False
133005n-c            -0.000     False
121910e-c            -0.000     True
134103n-n            -0.000     False
9393n-c              -0.000     True
13765c-e             -0.000     True
13765c-c             -0.000     True
98621n-c             -

{"664-e": -0.85, "664-n": -0.13, "664-c": -0.02, "1265-n": -0.41, "1552-n": -0.61, "1552-c": -0.22, "285-n": -0.8, "515-c": -0.62, "1474-n": -0.31, "1474-c": -0.48, "1436-e": -0.68, "1436-n": -0.28, "1338-n": -0.67, "362-e": -0.28, "362-n": -0.67, "753-n": -0.05, "753-c": -0.6, "110-n": -0.43, "110-c": -0.17, "1456-e": -0.72, "1456-n": -0.18, "1456-c": -0.1, "248-e": -0.7, "248-n": -0.25, "531-n": -0.79, "209-e": -0.56, "209-n": -0.31, "975-e": -0.72, "975-n": -0.28, "645-e": -0.75, "645-n": -0.16, "645-c": -0.09, "1592-n": -0.69, "561-e": -0.86, "561-c": -0.12, "1010-n": -0.29, "1010-c": -0.6, "10-e": -0.76, "10-n": -0.24, "1457-e": -0.67, "1457-n": -0.3, "1384-n": -0.32, "1384-c": -0.25, "595-e": -0.77, "595-n": -0.23, "1549-e": -0.72, "1549-n": -0.26, "76-e": -0.83, "76-c": -0.04, "64-n": -0.28, "64-c": -0.7, "1031-n": -0.32, "1031-c": -0.05, "494-n": -0.65, "398-e": -0.48, "398-n": -0.49, "428-e": -0.88, "428-n": -0.12, "608-n": -0.4, "608-c": -0.46, "973-n": -0.7, "1472-e": -0.03, "1472-n": -0.59, "1472-c": -0.38, "879-e": -0.49, "879-n": -0.43, "836-e": -0.28, "836-n": -0.67, "1568-e": -0.52, "1568-n": -0.47, "451-n": -0.47, "1595-n": -0.63, "474-n": -0.72, "163-n": -0.36, "1401-n": -0.47, "1362-e": -0.81, "1362-n": -0.19, "291-e": -0.9, "9-n": -0.4, "688-c": -0.56, "578-c": -0.91, "406-e": -0.86, "999-e": -0.69, "999-n": -0.31, "480-n": -0.41, "480-c": -0.55, "540-n": -0.75, "1165-n": -0.66, "1165-c": -0.25, "582-e": -0.72, "582-n": -0.25, "734-e": -0.81, "734-n": -0.17, "1379-n": -0.37, "2-e": -0.48, "2-n": -0.12, "2-c": -0.4, "861-n": -0.41, "1559-n": -0.15, "754-c": -0.89, "165-e": -0.9, "1588-e": -0.47, "1588-n": -0.48, "1588-c": -0.05, "525-n": -0.7, "525-c": -0.06, "1132-n": -0.54, "1132-c": -0.41, "1328-n": -0.34, "1328-c": -0.43, "942-c": -0.87, "1170-n": -0.78, "394-n": -0.88, "719-n": -0.14, "719-c": -0.07, "588-e": -0.64, "588-n": -0.36, "272-n": -0.65, "681-n": -0.52, "681-c": -0.41, "1319-e": -0.77, "1319-n": -0.22, "749-n": -0.63, "204-e": -0.56, "204-n": -0.41, "174-n": -0.54, "174-c": -0.05, "714-e": -0.45, "714-n": -0.52, "418-n": -0.42, "418-c": -0.19, "469-n": -0.16, "469-c": -0.63, "1349-e": -0.49, "1349-n": -0.5, "732-e": -0.91, "730-n": -0.64, "730-c": -0.21, "1493-n": -0.49, "1493-c": -0.32, "921-n": -0.72, "391-e": -0.75, "391-n": -0.24, "391-c": -0.01, "1274-e": -0.1, "1274-n": -0.2, "1274-c": -0.7, "680-e": -0.38, "470-n": -0.54, "195-e": -0.41, "195-n": -0.22, "782-c": -0.56, "267-e": -0.74, "267-n": -0.22, "185-e": -0.83, "534-e": -0.56, "534-n": -0.39, "134-e": -0.64, "134-c": -0.13, "252-e": -0.75, "996-e": -0.71, "996-n": -0.27, "1562-e": -0.21, "1562-n": -0.68, "155-n": -0.38, "155-c": -0.06, "331-n": -0.13, "331-c": -0.8, "696-n": -0.83, "696-c": -0.01, "703-e": -0.47, "703-n": -0.49, "799-e": -0.28, "799-n": -0.5, "789-e": -0.82, "886-e": -0.46, "886-n": -0.51, "388-n": -0.67, "1013-e": -0.66, "1013-n": -0.09, "1013-c": -0.25, "883-e": -0.07, "883-n": -0.8, "1185-e": -0.44, "1185-n": -0.54, "1144-e": -0.43, "1144-n": -0.57, "860-e": -0.34, "860-n": -0.52, "433-e": -0.77, "433-n": -0.19, "1090-e": -0.75, "1090-n": -0.24, "7-n": -0.16, "7-c": -0.73, "296-e": -0.49, "296-n": -0.48, "633-n": -0.42, "633-c": -0.54, "963-e": -0.85, "963-n": -0.14, "946-e": -0.4, "946-n": -0.22, "946-c": -0.38, "1249-n": -0.76, "747-n": -0.83, "1139-c": -0.69, "1415-e": -0.69, "1415-n": -0.27, "478-n": -0.64, "478-c": -0.3, "700-n": -0.17, "700-c": -0.51, "708-n": -0.55, "708-c": -0.11, "1507-n": -0.7, "964-e": -0.82, "964-n": -0.18, "1135-e": -0.57, "1135-n": -0.32, "1470-e": -0.77, "1470-n": -0.1, "1470-c": -0.13, "467-e": -0.78, "467-n": -0.21, "1175-e": -0.13, "1175-n": -0.48, "666-n": -0.52, "666-c": -0.16, "606-e": -0.65, "606-n": -0.32, "1589-n": -0.3, "1589-c": -0.68, "145-e": -0.2, "145-c": -0.61, "1532-e": -0.28, "1532-n": -0.55, "1220-n": -0.6, "1220-c": -0.3, "1019-n": -0.72, "1019-c": -0.16, "641-e": -0.12, "641-n": -0.72, "527-e": -0.41, "527-n": -0.56, "1277-e": -0.89, "1277-n": -0.08, "1277-c": -0.03, "450-n": -0.66, "685-n": -0.45, "685-c": -0.48, "164-n": -0.45, "164-c": -0.3, "1042-e": -0.97, "1072-c": -0.84, "1268-e": -0.7, "1268-n": -0.28, "501-n": -0.13, "501-c": -0.8, "1036-n": -0.69, "1125-e": -0.63, "1125-n": -0.32, "386-e": -0.69, "386-n": -0.3, "1252-e": -0.58, "1252-n": -0.41, "335-e": -0.71, "335-n": -0.22, "1407-c": -0.42, "699-e": -0.09, "699-c": -0.75, "1284-n": -0.63, "355-n": -0.49, "355-c": -0.5, "225-e": -0.7, "225-n": -0.22, "1427-n": -0.7, "1292-n": -0.38, "1292-c": -0.57, "95-n": -0.76, "581-e": -0.9, "581-n": -0.08, "691-e": -0.49, "691-n": -0.45, "893-e": -0.88, "893-n": -0.12, "438-e": -0.76, "780-e": -0.82, "780-n": -0.15, "1025-n": -0.07, "1025-c": -0.9, "118-e": -0.84, "118-n": -0.08, "169-e": -0.72, "1149-e": -0.06, "1149-n": -0.71, "1149-c": -0.23, "1380-n": -0.71, "974-e": -0.53, "974-n": -0.42, "716-n": -0.7, "1428-n": -0.86, "1536-n": -0.2, "1536-c": -0.17, "1006-e": -0.38, "1006-n": -0.56, "1221-n": -0.37, "1221-c": -0.61, "1193-n": -0.46, "1193-c": -0.43, "493-e": -0.65, "493-n": -0.19, "395-e": -0.68, "395-n": -0.27, "1376-e": -0.72, "1376-n": -0.28, "1060-e": -0.74, "1060-c": -0.04, "234-c": -0.39, "328-e": -0.85, "328-n": -0.15, "1190-e": -0.53, "1190-n": -0.39, "424-n": -0.45, "424-c": -0.31, "430-n": -0.53, "430-c": -0.44, "1029-e": -0.81, "67-e": -0.27, "67-n": -0.64, "67-c": -0.09, "343-e": -0.83, "343-n": -0.16, "1104-n": -0.24, "1104-c": -0.74, "114-n": -0.38, "1321-n": -0.24, "1321-c": -0.76, "807-n": -0.62, "392-n": -0.59, "392-c": -0.33, "1016-e": -0.8, "1016-n": -0.18, "1098-e": -0.56, "1098-n": -0.44, "456-n": -0.44, "456-c": -0.44, "140-e": -0.6, "140-n": -0.2, "771-e": -0.87, "1577-n": -0.55, "316-n": -0.47, "316-c": -0.52, "241-e": -0.69, "241-n": -0.28, "1297-c": -0.64, "825-n": -0.79, "825-c": -0.17, "1580-n": -0.64, "1197-e": -0.74, "563-e": -0.54, "563-n": -0.43, "1069-e": -0.19, "1069-n": -0.52, "1069-c": -0.29, "321-e": -0.88, "321-n": -0.07, "321-c": -0.05, "466-n": -0.35, "466-c": -0.52, "609-n": -0.63, "738-e": -0.72, "738-n": -0.24, "536-e": -0.56, "536-n": -0.41, "557-e": -0.33, "557-n": -0.51, "557-c": -0.16, "1550-n": -0.34, "40-n": -0.64, "40-c": -0.3, "1232-n": -0.38, "1232-c": -0.51, "885-e": -0.65, "885-n": -0.3, "208-e": -0.43, "208-n": -0.33, "208-c": -0.24, "125-n": -0.48, "125-c": -0.51, "1537-n": -0.91, "309-c": -0.47, "591-e": -0.44, "591-n": -0.54, "119-n": -0.54, "119-c": -0.45, "369-n": -0.67, "369-c": -0.31, "1244-c": -0.63, "1471-n": -0.79, "1471-c": -0.2, "1313-n": -0.7, "1070-e": -0.64, "1070-n": -0.35, "770-n": -0.68, "770-c": -0.3, "117-n": -0.4, "117-c": -0.57, "1166-n": -0.43, "1166-c": -0.54, "271-e": -0.69, "271-n": -0.29, "107-e": -0.91, "1539-n": -0.57, "1078-e": -0.84, "1078-c": -0.0, "273-n": -0.54, "1178-e": -0.78, "158-n": -0.11, "1015-n": -0.73, "374-n": -0.54, "374-c": -0.46, "890-e": -0.84, "890-n": -0.15, "824-e": -0.03, "824-n": -0.44, "871-e": -0.23, "871-c": -0.68, "1158-n": -0.52, "51-c": -0.76, "1448-e": -0.88, "287-e": -0.88, "287-c": -0.01, "1486-e": -0.83, "1486-n": -0.15, "553-n": -0.34, "553-c": -0.52, "487-n": -0.36, "839-e": -0.46, "839-c": -0.16, "1259-e": -0.66, "1259-n": -0.26, "652-n": -0.6, "1219-e": -0.31, "1219-n": -0.3, "1219-c": -0.39, "723-e": -0.99, "779-e": -0.6, "779-n": -0.32, "13-n": -0.57, "13-c": -0.42, "607-e": -0.74, "607-n": -0.26, "1273-n": -0.74, "863-n": -0.55, "863-c": -0.35, "972-e": -0.61, "972-n": -0.34, "972-c": -0.05, "776-e": -0.46, "776-n": -0.46, "454-n": -0.37, "454-c": -0.59, "528-n": -0.48, "1368-e": -0.52, "1368-n": -0.46, "1107-c": -0.42, "743-e": -0.12, "743-n": -0.11, "743-c": -0.77, "481-n": -0.5, "481-c": -0.26, "649-n": -0.63, "649-c": -0.31, "737-e": -0.77, "737-n": -0.21, "737-c": -0.02, "347-e": -0.73, "347-n": -0.26, "229-e": -0.03, "229-n": -0.33, "229-c": -0.64, "1323-n": -0.44, "593-n": -0.59, "593-c": -0.33, "900-n": -0.49, "303-e": -0.56, "303-c": -0.43, "1445-n": -0.11, "1445-c": -0.87, "262-e": -0.49, "262-n": -0.47, "1458-n": -0.67, "1458-c": -0.19, "1202-n": -0.68, "1497-n": -0.65, "1497-c": -0.33, "1320-e": -0.9, "1320-n": -0.09, "864-e": -0.76, "864-n": -0.24, "477-n": -0.7, "57-n": -0.68, "434-e": -0.24, "434-n": -0.59, "434-c": -0.17, "298-n": -0.79, "215-n": -0.2, "215-c": -0.57, "872-e": -0.63, "872-n": -0.37, "682-e": -0.36, "682-c": -0.17, "1502-e": -0.28, "1502-n": -0.59, "752-n": -0.57, "643-e": -0.68, "643-n": -0.32, "1092-c": -0.56, "600-e": -0.63, "600-n": -0.37, "859-c": -0.76, "986-e": -0.45, "986-n": -0.51, "1394-e": -0.2, "1394-n": -0.38, "1394-c": -0.42, "1386-n": -0.47, "1386-c": -0.5, "787-c": -0.63, "923-e": -0.6, "923-c": -0.05, "461-n": -0.66, "461-c": -0.12, "618-n": -0.39, "618-c": -0.41, "420-e": -0.88, "420-c": -0.12, "1467-e": -0.9, "375-e": -0.61, "375-n": -0.38, "1155-e": -0.79, "1155-n": -0.16, "280-n": -0.73, "370-c": -0.31, "880-e": -0.71, "880-n": -0.15, "880-c": -0.14, "202-n": -0.65, "1499-n": -0.84, "1499-c": -0.11, "1101-e": -0.36, "1101-n": -0.28, "1101-c": -0.36, "961-e": -0.78, "961-n": -0.22, "961-c": -0.0, "1326-e": -0.52, "1326-n": -0.44, "506-e": -0.76, "325-e": -0.39, "325-n": -0.47, "853-e": -0.3, "853-n": -0.54, "509-e": -0.88, "1134-n": -0.62, "1134-c": -0.38, "1124-n": -0.67, "1317-n": -0.23, "1317-c": -0.17, "1228-e": -0.66, "1228-n": -0.34, "1228-c": -0.0, "1333-e": -0.55, "1333-n": -0.39, "71-e": -0.79, "71-n": -0.15, "71-c": -0.06, "245-e": -0.07, "245-c": -0.64, "654-n": -0.41, "579-e": -0.57, "584-e": -0.74, "584-n": -0.21, "795-e": -0.62, "795-n": -0.36, "1382-n": -0.8, "905-n": -0.73, "567-n": -0.48, "1180-e": -0.35, "1180-n": -0.23, "854-e": -0.74, "854-n": -0.24, "1555-n": -0.3, "1216-n": -0.7, "1216-c": -0.23, "521-e": -0.67, "521-n": -0.31, "990-n": -0.26, "419-e": -0.64, "419-n": -0.28, "1538-e": -0.68, "910-e": -0.59, "910-n": -0.37, "360-n": -0.25, "360-c": -0.24, "901-e": -0.53, "901-n": -0.47, "1097-n": -0.36, "1097-c": -0.47, "1540-e": -0.62, "1540-n": -0.33, "1540-c": -0.05, "1200-e": -0.56, "1200-n": -0.42, "233-e": -0.86, "1086-c": -0.58, "447-n": -0.31, "447-c": -0.57, "729-e": -0.66, "729-n": -0.31, "674-n": -0.44, "674-c": -0.55, "706-e": -0.75, "1430-n": -0.64, "1262-n": -0.62, "25-n": -0.14, "25-c": -0.76, "1056-e": -0.54, "1056-n": -0.44, "1236-n": -0.44, "1236-c": -0.13, "136-e": -0.64, "136-c": -0.11, "1441-e": -0.88, "1441-n": -0.12, "48-e": -0.6, "48-n": -0.36, "401-n": -0.2, "401-c": -0.28, "1131-n": -0.5, "1131-c": -0.49, "731-e": -0.58, "731-n": -0.38, "449-e": -0.66, "449-n": -0.34, "1204-n": -0.45, "1204-c": -0.15, "311-e": -0.43, "311-n": -0.2, "311-c": -0.37, "1129-e": -0.88, "123-c": -0.89, "585-e": -0.76, "585-n": -0.22, "585-c": -0.02, "513-n": -0.47, "513-c": -0.5, "718-e": -0.78, "718-n": -0.15, "718-c": -0.07, "761-n": -0.5, "1142-e": -0.42, "1142-n": -0.4, "684-e": -0.59, "684-n": -0.33, "684-c": -0.08, "42-e": -0.6, "42-n": -0.37, "112-e": -0.46, "112-n": -0.43, "368-n": -0.14, "368-c": -0.85, "278-n": -0.74, "278-c": -0.13, "1030-e": -0.43, "1030-n": -0.36, "1030-c": -0.21, "896-e": -0.7, "896-n": -0.28, "815-n": -0.16, "815-c": -0.07, "724-n": -0.4, "1066-e": -0.2, "1066-n": -0.7, "629-e": -0.86, "1437-n": -0.36, "1437-c": -0.04, "181-n": -0.78, "1361-e": -0.61, "1361-c": -0.02, "894-c": -0.85, "1018-n": -0.65, "1018-c": -0.33, "1251-e": -0.64, "1251-n": -0.31, "1396-e": -0.62, "1396-n": -0.35, "415-e": -0.28, "415-c": -0.66, "605-n": -0.45, "1513-n": -0.49, "1513-c": -0.4, "354-n": -0.68, "354-c": -0.31, "926-n": -0.45, "926-c": -0.52, "755-e": -0.68, "755-n": -0.31, "1373-n": -0.64, "307-c": -0.79, "1138-e": -0.73, "1138-n": -0.22, "446-e": -0.65, "446-n": -0.34, "345-e": -0.24, "345-n": -0.74, "532-n": -0.8, "62-e": -0.63, "62-n": -0.37, "104-e": -0.5, "104-c": -0.14, "1064-n": -0.91, "282-n": -0.31, "282-c": -0.12, "246-c": -0.76, "471-c": -0.77, "788-n": -0.8, "884-n": -0.48, "884-c": -0.48, "315-n": -0.49, "315-c": -0.43, "310-n": -0.45, "937-n": -0.77, "937-c": -0.08, "365-e": -0.49, "365-n": -0.5, "222-e": -0.62, "222-n": -0.36, "4-e": -0.68, "4-n": -0.29, "1566-c": -0.91, "75-e": -0.33, "75-n": -0.57, "1383-e": -0.81, "1383-n": -0.18, "1161-n": -0.45, "837-n": -0.25, "835-e": -0.19, "835-n": -0.18, "835-c": -0.63, "850-e": -0.75, "850-c": -0.03, "1548-e": -0.67, "1548-n": -0.29, "486-e": -0.72, "486-n": -0.26, "161-n": -0.78, "152-n": -0.52, "152-c": -0.47, "1356-e": -0.6, "1356-n": -0.36, "336-n": -0.56, "336-c": -0.1, "750-e": -0.78, "750-n": -0.18, "750-c": -0.04, "523-n": -0.48, "523-c": -0.12, "526-e": -0.76, "526-n": -0.22, "512-n": -0.66, "1525-e": -0.61, "1525-n": -0.28, "284-n": -0.16, "284-c": -0.62, "520-e": -0.69, "520-n": -0.29, "120-e": -0.72, "292-n": -0.65, "783-e": -0.72, "783-n": -0.26, "783-c": -0.02, "1231-e": -0.5, "1231-n": -0.42, "275-n": -0.47, "920-e": -0.27, "920-c": -0.61, "1077-n": -0.41, "1479-e": -0.44, "1479-n": -0.54, "1503-n": -0.66, "1503-c": -0.13, "1150-e": -0.88, "1150-c": -0.03, "865-n": -0.4, "1057-e": -0.75, "1057-n": -0.22, "800-e": -0.73, "800-n": -0.25, "1345-e": -0.81, "774-e": -0.52, "774-n": -0.45, "976-n": -0.7, "976-c": -0.23, "720-n": -0.3, "720-c": -0.47, "556-n": -0.62, "1133-n": -0.38, "1133-c": -0.4, "1241-e": -0.72, "1241-n": -0.25, "1241-c": -0.03, "137-n": -0.7, "1584-e": -0.81, "1584-n": -0.16, "914-e": -0.86, "914-n": -0.14, "1341-e": -0.04, "1341-n": -0.56, "1176-e": -0.1, "1176-n": -0.52, "1267-e": -0.93, "1267-n": -0.05, "346-e": -0.78, "346-n": -0.18, "1596-n": -0.52, "1596-c": -0.41, "827-e": -0.6, "827-n": -0.14, "827-c": -0.26, "943-e": -0.67, "943-n": -0.31, "1230-n": -0.45, "806-n": -0.74, "221-e": -0.74, "648-n": -0.69, "648-c": -0.28, "1270-n": -0.37, "1270-c": -0.61, "1048-n": -0.56, "1048-c": -0.08, "1350-e": -0.43, "1350-n": -0.54, "1564-n": -0.78, "844-e": -0.88, "844-n": -0.12, "1001-e": -0.51, "1001-n": -0.45, "496-e": -0.75, "496-n": -0.18, "409-e": -0.78, "409-n": -0.21, "265-n": -0.46, "265-c": -0.53, "1208-e": -0.81, "1208-n": -0.19, "1208-c": -0.0, "1298-e": -0.75, "1298-n": -0.17, "1298-c": -0.08, "23-e": -0.46, "23-n": -0.45, "638-e": -0.09, "638-n": -0.6, "638-c": -0.31, "1500-e": -0.47, "1500-n": -0.5, "704-e": -0.69, "704-n": -0.26, "704-c": -0.05, "612-e": -0.81, "612-n": -0.19, "612-c": -0.0, "356-n": -0.54, "356-c": -0.35, "1567-e": -0.68, "1567-n": -0.31, "93-e": -0.78, "93-n": -0.15, "945-e": -0.66, "945-n": -0.27, "1266-n": -0.54, "1266-c": -0.39, "530-c": -0.72, "170-e": -0.84, "170-n": -0.16, "1163-n": -0.77, "1095-e": -0.37, "1095-n": -0.5, "427-e": -0.67, "427-n": -0.33, "1196-e": -0.92, "22-c": -0.69}